In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Prepare Data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [3]:
# Model
# f = w.x + b then apply sigmoid at the end
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        sig = nn.Sigmoid()
        y_pred = sig(self.linear(x))
        return y_pred

In [4]:
X_train.shape

torch.Size([455, 30])

In [5]:
n_samples, n_features = X_train.shape
learning_rate = 0.01

model = LogisticRegression(n_features)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#Training Loop
num_epoch=200
for epoch in range(num_epoch):
    # Forward Pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    
    # Backward Pass
    loss.backward()
    
    # update weights
    optimizer.step()
    optimizer.zero_grad()
    
    if (epoch+1)%10 == 0:
        [w, b] = model.parameters()
        print(f'epoch {epoch+1}: w = {w[0][0].item():.3f}, loss = {loss.item():.8f}')

epoch 10: w = -0.060, loss = 0.50767487
epoch 20: w = -0.081, loss = 0.42974487
epoch 30: w = -0.099, loss = 0.37809482
epoch 40: w = -0.113, loss = 0.34113038
epoch 50: w = -0.126, loss = 0.31317335
epoch 60: w = -0.138, loss = 0.29114604
epoch 70: w = -0.149, loss = 0.27324146
epoch 80: w = -0.158, loss = 0.25832886
epoch 90: w = -0.167, loss = 0.24566384
epoch 100: w = -0.175, loss = 0.23473530
epoch 110: w = -0.183, loss = 0.22518082
epoch 120: w = -0.190, loss = 0.21673530
epoch 130: w = -0.197, loss = 0.20920004
epoch 140: w = -0.204, loss = 0.20242287
epoch 150: w = -0.210, loss = 0.19628543
epoch 160: w = -0.216, loss = 0.19069393
epoch 170: w = -0.221, loss = 0.18557288
epoch 180: w = -0.226, loss = 0.18086070
epoch 190: w = -0.231, loss = 0.17650646
epoch 200: w = -0.236, loss = 0.17246863


In [6]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'Accuracy = {acc:.4f}')

Accuracy = 0.9298
